## Map/Reduce Semantics and Systems

### Types and transformations
* Map is a transformation
  * Input domain to output domain
* Reduce is a collection
  * No domain change
  
$$
map (k1, v1) \rightarrow list(k2,v2) \\
reduce (k2, list(v2)) \rightarrow list(v2)
$$

* Google C++ implementation is based all on strings
  * User code must convert to structured types
* Hadoop! has type wrappers

### Parallelism in Map/Reduce

* How much potential paralleism in mappers? in reducers?


.

.

......spoiler alert.......
  
.  

.
* Mappers: up to a parallel process for each input (typically a file)
* Reducers: up to a paralle process for each key
* So for the WordCount example
  * two files = two mappers
  * 5 different words = five mappers
  * but this is scalable with input


Differentiating betweens mapper/reducer and map/reduce processes
* A cluster will typically configure number of available phyical processes
  * this numer is typically much smaller than potential parallelism
  * we refer to `mappers` as potential parallelism and `map processes` as the number of phyical processes running map functions.

### Map/Reduce Runtimes

From the Google paper https://www.usenix.org/legacy/events/osdi04/tech/dean.html

<img src="images/mr.png" width=512 />

* Automatically partition input data
  * 16-64 MB chunks for Google
* Create M map tasks: one for each chunk
  * Assign available workers (up to M) to tasks
* Write intermediate pairs to local (to worker) disk
* R reduce tasks (defined by user) read and process intermediate results
* Output is up to R files available on shared file system
* Master tracks state
  * Asssignment of M map tasks and R reduce tasks to workers
  * State and liveliness of the above
